In [ ]:
!pip install -q torch_xla[tpu] -f https://storage.googleapis.com/libtorch-xla-releases/wheels/tpuvm/colab.html # Only if you wanna use tpus on colab

# CIFAR-10/KMEANS paper/ResNET-18 (ignore this; older)

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.models import resnet18
from torch.optim.lr_scheduler import CosineAnnealingLR
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
# from sklearn.cluster import KMeans
from tqdm import tqdm
from torch.utils.data import Subset

import torch_xla
import torch_xla.core.xla_model as xm
import torch_xla.distributed.parallel_loader as pl

"""Dataset part"""

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])
train_dataset = datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root="./data", train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=10000, shuffle=True)
val_loader = DataLoader(test_dataset, batch_size=20, shuffle=False)


mean_image = 0.0
total_samples = 0
denom = 0.0

for batch_idx, (inputs, targets) in enumerate(tqdm(train_loader, desc="Loading CIFAR-10")):
    batch_samples = inputs.size(0)
    mean_image += inputs.sum(dim=0)  # sum over batch dimension -> shape (C, H, W)
    total_samples += batch_samples

mean_image /= total_samples
mu_flat = mean_image.view(1, -1)

for batch_idx, (inputs, targets) in enumerate(tqdm(train_loader, desc="Computing denominator")):
    # inputs: shape (B, C, H, W)
    batch_flat = inputs.view(inputs.size(0), -1)

    # Compute squared distance to mean for each sample in batch
    dists_squared = ((batch_flat - mu_flat) ** 2).sum(dim=1)

    # Sum
    denom += dists_squared.sum().item()

q_values = torch.empty(total_samples) # Our q(x)
start_idx = 0

for batch_idx, (inputs, targets) in enumerate(tqdm(train_loader, desc="Computing q(x) for all dataset indices")):
    batch_size = inputs.size(0)
    end_idx = start_idx + batch_size

    # Compute squared distances to the mean
    batch_flat = inputs.view(batch_size, -1)
    dists_squared = ((batch_flat - mu_flat) ** 2).sum(dim=1)

    # Compute q(x)
    q_batch = 0.5 * (1 / total_samples) + 0.5 * (dists_squared / denom)
    q_values[start_idx:end_idx] = q_batch

    start_idx = end_idx

KeyboardInterrupt: 

In [ ]:
# 1 / q(x)
sampling_probs = (1.0 / q_values)
sampling_probs /= sampling_probs.sum()  # normalize to sum to 1

m = 20000  # TODO Use the general way later
sample_indices = torch.multinomial(sampling_probs, num_samples=m, replacement=False)

coreset = Subset(train_dataset, sample_indices.tolist())
coreset_loader = torch.utils.data.DataLoader(coreset, batch_size=2048, shuffle=False)

In [ ]:
"""Training model part"""

# Use MPS if available (for Macs), otherwise fallback
# device = torch.device("mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu")
device = xm.xla_device()
print(f"Using device: {device}")

# Load ResNet18

model = resnet18(num_classes=10).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)
scheduler = CosineAnnealingLR(optimizer, T_max=300)

# Training loop
def train(model, train_loader, epochs=300):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        para_loader = pl.MpDeviceLoader(train_loader, device)
        loop = tqdm(para_loader, desc=f"Epoch [{epoch+1}/{epochs}]")

        for inputs, targets in loop:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            xm.optimizer_step(optimizer)

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            loop.set_postfix(loss=running_loss/(total/inputs.size(0)), acc=100.*correct/total)

        scheduler.step()
    return model

# Validation loop
def validate(model, val_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        para_loader = pl.MpDeviceLoader(val_loader, device)
        for inputs, targets in para_loader:
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    acc = 100. * correct / total
    print(f"Validation Accuracy: {acc:.2f}%")

# Main entry
model = train(model, coreset_loader, epochs=35)
validate(model, val_loader)

Using device: xla:0


Epoch [35/35]: 100%|██████████| 10/10 [00:04<00:00,  2.38it/s, acc=100, loss=0.00113]


Validation Accuracy: 52.75%


# Forests/Kmeans paper/RBFNN paper/kmeans++

In [ ]:
from sklearn.datasets import fetch_covtype
from sklearn.preprocessing import MinMaxScaler

# Load the data
X, y = fetch_covtype(return_X_y=True)

print(f"Original Shape of X: {X.shape}")

# Normalize X to be between 0 and 1
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

print(f"After normalization, min value: {X.min()}, max value: {X.max()}")

Original Shape of X: (581012, 54)
After normalization, min value: 0.0, max value: 1.0


## Lightweight coreset

In [ ]:
import numpy as np

class LightweightCoreset:
    def __init__(self, X, k, eps):
        self.X = X
        self.k = k
        self.eps = eps

    def set_k(self, k):
        self.k = k

    def set_X(self, X):
        self.X = X

    def _compute_m(self):
        #Computing coreset size
        self.m = np.int64(self.X.shape[1]*self.k*np.log2(self.k)/np.power(self.eps, 2))
        if self.m > self.X.shape[0]*0.2:
            self.m = int(self.k * 0.01 * self.X.shape[0])

    def _compute_coreset(self):
        #Algorithm 1 Lightweight coreset construction
        dist = np.power(self.X-self.X.mean(axis=0), 2).sum(axis=1)
        q = 0.5/self.X.shape[0] + 0.5*dist/dist.sum()
        indices = np.random.choice(self.X.shape[0], size=self.m, replace=True)
        X_cs = self.X[indices, :]
        w_cs = 1.0/(self.m*q[indices])
        return X_cs, w_cs

    def compute(self):
        self._compute_m()
        print("coreset size: ", self.m)
        return self._compute_coreset()

## RBFNN coreset

In [ ]:
import numpy as np
from scipy.optimize import approx_fprime
import scipy as sp
import matplotlib.pyplot as plt
from scipy.spatial import ConvexHull
from scipy.io import loadmat
import time
import numpy.linalg as la
import copy
from mpl_toolkits import mplot3d
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import torch.nn.functional as F
import matplotlib.pyplot as plt


np.seterr(all='raise')


class MVEEApprox(object):
    Epsilon = 1e-6

    def __init__(self, P, cost_func, maxIter=10, bound=1):
        self.cost_func = cost_func
        self.P = P
        self.bound = bound
        self.maxIter = maxIter
        self.c = np.zeros((P.shape[1], ))
        self.G = max(np.sqrt(P.shape[0]), np.max(np.sum(np.abs(P)**2,axis=-1)**(0.5))) * np.eye(P.shape[1], P.shape[1])
        self.oldG = copy.deepcopy(self.G)


    def separation_oracle(self, x):
        # grad = approx_fprime(x, self.cost_func, self.Epsilon)
        grad = self.P.T.dot(np.sign(self.P.dot(x)))
        return grad / np.linalg.norm(grad, np.inf)

    def get_axis_points(self):
        if np.any(np.isnan(self.G)):
            raise ValueError('WHATT!!!!')
        U, s, vh = np.linalg.svd(self.G, full_matrices=True)
        # volume = np.prod(np.sqrt(s))
        d = s.shape[0]
        A = np.dot(np.diag(np.sqrt(s) / np.sqrt(d + 1)), U.T)
        points = np.vstack((A, -A))
        # points = np.tile(, vh.T)), d, axis=0)
        # temp = np.repeat(np.vstack((self.c[:, np.newaxis].T, -self.c[:, np.newaxis].T)), d, axis=0)
        temp = np.tile(self.c[:, np.newaxis].T, (2*d, 1))
        return points + temp

    def check_if_inside(self, P):
        #vals = np.apply_along_axis(self.cost_func, 1, P)
        vals = np.linalg.norm(P.dot(self.P.T), ord=1, axis=1)
        i = np.argmax(vals, axis=0)
        if vals[i] <= 1:
            return np.inf, vals[i]

        print('Maximal Value: {:.4f}'.format(vals[i]))

        return i, vals[i]

    def basic_ellipsoid_method(self):
        d = np.ma.size(self.P, axis=1)

        self.oldG = copy.deepcopy(self.G)
        while self.cost_func(self.c) > 1 :
            H = self.separation_oracle(self.c)
            b = np.dot(self.G, H) / np.sqrt(np.abs(np.dot(H, np.dot(self.G, H))))
            self.c = self.c - 1.0 / (d + 1.0) * b
            self.G = d ** 2.0 / (d ** 2.0 - 1.0) * (self.G - (2.0 / (d + 1.0)) * np.dot(b[:, np.newaxis], b[:, np.newaxis].T))

        if not self.isPD(self.G):
            print('Corrected back to PSD at Basic ellipsoid method')
            self.G = self.nearestPD(self.G)


    def shallow_cut_update(self, point):
        d = np.ma.size(self.G, 0)
        rho = 1.0 / (d + 1.0) ** 2.0
        sigma = d ** 3.0 * (d + 2.0) / ((d + 1) ** 3.0 * (d - 1.0))
        zeta = 1.0 + 1.0 / (2.0 * d ** 2.0 * (d + 1.0) ** 2.0)
        tau = 2.0 / ((d + 1.0) * d)

        b = np.dot(self.G, point) / np.sqrt(np.abs(np.dot(point, np.dot(self.G, point))))
        self.oldG = copy.deepcopy(self.G)
        self.G = zeta * sigma * (self.G - tau * np.dot(b[:, np.newaxis], b[:, np.newaxis].T))
        self.c = self.c - rho * b

        if not self.isPD(self.G):
            print('Corrected back to PSD at Shallow cut update')
            self.G = self.nearestPD(self.G)

    def compute_approximated_MVEE(self):
        stop = False
        iter = 0
        while not stop:
            s = time.time()
            self.basic_ellipsoid_method()
            print(time.time() - s)

            s = time.time()
            axis_points = self.get_axis_points()
            print(time.time() - s)
            i, val = self.check_if_inside(axis_points)
            if np.isinf(i):
                stop = True
            else:
                sep_grad = self.separation_oracle(axis_points[i, :])
                self.shallow_cut_update(sep_grad)

            if iter > self.maxIter:
                self.G = self.G / val
                iter = 0
                print('HMM')
                continue
            iter += 1

        E = np.linalg.cholesky(self.G)
        return E, self.c

    @staticmethod
    def nearestPD(A):
        """Find the nearest positive-definite matrix to input

        A Python/Numpy port of John D'Errico's nearestSPD MATLAB code [1], which
        credits [2].

        [1] https://www.mathworks.com/matlabcentral/fileexchange/42885-nearestspd

        [2] N.J. Higham, "Computing a nearest symmetric positive semidefinite
        matrix" (1988): https://doi.org/10.1016/0024-3795(88)90223-6
        """

        B = (A + A.T) / 2
        _, s, V = la.svd(B)

        H = np.dot(V.T, np.dot(np.diag(s), V))

        A2 = (B + H) / 2

        A3 = (A2 + A2.T) / 2

        if MVEEApprox.isPD(A3):
            return A3

        spacing = np.spacing(la.norm(A))
        # The above is different from [1]. It appears that MATLAB's chol Cholesky
        # decomposition will accept matrixes with exactly 0-eigenvalue, whereas
        # Numpy's will not. So where [1] uses eps(mineig) (where eps is Matlab
        # for np.spacing), we use the above definition. CAVEAT: our spacing
        # will be much larger than [1]'s eps(mineig), since mineig is usually on
        # the order of 1e-16, and eps(1e-16) is on the order of 1e-34, whereas
        # spacing will, for Gaussian random matrixes of small dimension, be on
        # othe order of 1e-16. In practice, both ways converge, as the unit test
        # below suggests.
        I = np.eye(A.shape[0])
        k = 1
        while not MVEEApprox.isPD(A3):
            mineig = np.min(np.real(la.eigvals(A3)))
            A3 += I * (-mineig * k ** 2 + spacing)
            k += 1

        return A3

    @staticmethod
    def isPD(B):
        """Returns true when input is positive-definite, via Cholesky"""
        try:
            _ = la.cholesky(B)
            return True
        except la.LinAlgError:
            return False

    @staticmethod
    def plotEllipsoid(center, radii, rotation, ax=None, plotAxes=True, cageColor='r', cageAlpha=1):
        """Plot an ellipsoid"""
        make_ax = (ax is None)
        if make_ax:
            fig = plt.figure()
            ax = fig.add_subplot(111, projection='3d')

        u = np.linspace(0.0, 2.0 * np.pi, 100)
        v = np.linspace(0.0, np.pi, 100)

        # cartesian coordinates that correspond to the spherical angles:
        x = radii[0] * np.outer(np.cos(u), np.sin(v))
        y = radii[1] * np.outer(np.sin(u), np.sin(v))
        z = radii[2] * np.outer(np.ones_like(u), np.cos(v))
        # rotate accordingly
        for i in range(len(x)):
            for j in range(len(x)):
                [x[i, j], y[i, j], z[i, j]] = np.dot(np.array([x[i, j], y[i, j], z[i, j]]), rotation) + center.flatten()

        if plotAxes:
            # make some purdy axes
            axes = np.array([[radii[0], 0.0, 0.0],
                             [0.0, radii[1], 0.0],
                             [0.0, 0.0, radii[2]]])
            # rotate accordingly
            for i in range(len(axes)):
                axes[i] = np.dot(axes[i], rotation)

            print('Axis are: ', axes)
            # print(axes + center.flatten())

            # plot axes
            print('Whole points are: ')
            for p in axes:
                X3 = np.linspace(-p[0], p[0], 2) + center[0]
                Y3 = np.linspace(-p[1], p[1], 2) + center[1]
                Z3 = np.linspace(-p[2], p[2], 2) + center[2]
                ax.plot3D(X3, Y3, Z3, color='m')
                PP = np.vstack((X3, Y3, Z3)).T
                print(PP)

        # plot ellipsoid
        ax.plot_wireframe(x, y, z, rstride=4, cstride=4, color=cageColor, alpha=cageAlpha)
        plt.show()

    def plotBodyAndEllips(self, B, E):
        N = 10000
        U, D, V = np.linalg.svd(E, full_matrices=True)
        a = D[0]
        b = D[1]
        theta = np.expand_dims(np.arange(start=0, step=1.0 / N, stop=2.0 * np.pi + 1.0 / N), 1).T

        state = np.vstack((a * np.cos(theta), b * np.sin(theta)))
        X = np.dot(U, state) + self.c[:, np.newaxis]

        ax = plt.subplot(111)
        plt.plot(X[0, :], X[1, :], color='black', linewidth=5)

        vals = np.apply_along_axis(lambda x: np.linalg.norm(x.flatten() - self.c.flatten()), 0, X)
        i = np.argmax(vals)

        print(X[:, i])

        plt.scatter(self.c[0], self.c[1], marker='+', color='green')
        plt.grid(True)

        # hull = ConvexHull(B)
        # for simplex in hull.simplices:
        #     plt.plot(B[simplex, 0], B[simplex, 1], 'k-')

        # plt.scatter(B[:, 0], B[:, 1], marker='D', color='orange')

        # plt.scatter(self.c[0], self.c[1], marker='^', color='green')
        # plt.scatter(X[0, i], X[1, i], marker='*', color='black')
        plt.scatter(B[:, 0], B[:, 1], marker='*', color='green')
        plt.show()

    @staticmethod
    def main():
        P = np.random.rand(10000, 400)
        cost_func = lambda x: np.linalg.norm(np.dot(P, x), ord=1)
        tol = 1/100
        start_time = time.time()

        mvee = MVEEApprox(P, cost_func, maxIter=10)
        E, C = mvee.compute_approximated_MVEE()

        print('Ellip took {:.4f}'.format(time.time() - start_time))
        if P.shape[1] <= 3:
            N = 1000
            X = np.random.randn(N, P.shape[1])
            vals = np.apply_along_axis(cost_func, 1, X)
            X = np.multiply(X, 1.0 / vals[:, np.newaxis])
            if P.shape[1] == 2:
                mvee.plotBodyAndEllips(X, E)
            else:
                fig = plt.figure()
                ax = plt.axes(projection='3d')

                # from scipy.spatial import ConvexHull
                # hull = ConvexHull(X)

                # Plot defining corner points
                # ax.plot(X.T[0], X.T[1], X.T[2], "ko")
                # for s in hull.simplices:
                #     s = np.append(s, s[0])  # Here we cycle back to the first coordinate
                #     ax.plot(X[s, 0], X[s, 1], X[s, 2], "b-")
                ax.scatter3D(X[:, 0], X[:, 1], X[:, 2], color='black', marker='o')
                U, D, V = la.svd(E, full_matrices=True)
                mvee.plotEllipsoid(C, D, U.T, ax=ax)

R = 10

def obtainSensitivity(X, w, approxMVEE=False):
    if not approxMVEE:
        raise Exception
    else:
        cost_func = lambda x: np.linalg.norm(np.dot(X, x), ord=1)
        mvee = MVEEApprox(X, cost_func, 3)
        ellipsoid, center = mvee.compute_approximated_MVEE()
        U = X.dot(ellipsoid)
        return np.linalg.norm(U, ord=1, axis=1)


def generateCoreset(X, y, sensitivity, sample_size, weights=None, SEED=1):
    if weights is None:
        weights = np.ones((X.shape[0], 1)).flatten()

    # Compute the sum of sensitivities.
    t = np.sum(sensitivity)

    # The probability of a point prob(p_i) = s(p_i) / t
    probability = sensitivity.flatten() / t

    startTime = time.time()

    # initialize new seed
    np.random.seed()

    # Multinomial Distribution
    hist = np.random.choice(np.arange(probability.shape[0]), size=sample_size, replace=False, p=probability.flatten())
    indxs, counts = np.unique(hist, return_counts=True)
    S = X[indxs]
    labels = y[indxs]

    # Compute the weights of each point: w_i = (number of times i is sampled) / (sampleSize * prob(p_i))
    weights = np.asarray(np.multiply(weights[indxs], counts), dtype=float).flatten()

    weights = np.multiply(weights, 1.0 / (probability[indxs] * sample_size))
    timeTaken = time.time() - startTime

    return indxs, S, labels, weights, timeTaken

## PCA

In [ ]:
from sklearn.decomposition import PCA
n_features = X.shape[1]
n_components = max(1, n_features // 10)  # At least 1 component
print(f"Number of PCA components: {n_components}")

# Perform PCA
pca = PCA(n_components=n_components)
X_reduced = pca.fit_transform(X)

Number of PCA components: 5


## Experiments

In [ ]:
## Lightweight coreset
lwcs = LightweightCoreset(X, 10, 0.1)
X_cs_lwc, w_cs_lwc = lwcs.compute()

coreset size:  58101


In [ ]:
%%capture
## RBFNN coreset
# ----- 1. scale to the unit ball (‖x‖₂ ≤ 1) ---------------------------------
R = np.max(np.linalg.norm(X, axis=1))
X_scaled = X / R            #  now every point satisfies ‖x‖₂ ≤ 1

# ----- 2. lifting step  q_p = [‖x‖² , -2xᵀ , 1] ------------------------------
phi = np.hstack([
    np.sum(X_scaled**2, axis=1, keepdims=True),   # ‖x‖₂²
    -2 * X_scaled,                                # -2 xᵀ
    np.ones((X_scaled.shape[0], 1))               # 1
])

# ----- 3. sensitivities & coreset -------------------------------------------
sens = obtainSensitivity(phi, w=None, approxMVEE=True)
m = 58101                                        # coreset size
idx, X_cs_rbfnn, labels, w_cs_rbfnn, _ = generateCoreset(phi, y, sens, m)
print(f"Coreset shape: {X_cs_rbfnn.shape}")
print(f"Coreset labels shape: {labels.shape}")
print(f"Coreset weights shape: {w_cs_rbfnn.shape}")

In [ ]:
import numpy as np
from sklearn.datasets import fetch_covtype
from sklearn.cluster import KMeans

def compute_quantization_error(X, centers):
    # Assign each point to the nearest center
    from scipy.spatial import distance
    labels = np.argmin(distance.cdist(X, centers), axis=1)
    # Squared distance from each point to its nearest center
    distances = np.linalg.norm(X - centers[labels], axis=1) ** 2
    return distances.sum()

def kmeans_plus_plus_on_corset(X_cs, w_cs, k):
    # Run kmeans++ on the weighted corset
    kmeans = KMeans(n_clusters=k, init='k-means++', random_state=42)
    kmeans.fit(X_cs, sample_weight=w_cs)
    centers = kmeans.cluster_centers_
    return centers

# Example usage:
k = 10
n_samples = X.shape[0]

# 1. Full dataset
kmeans_full = KMeans(n_clusters=k, init='k-means++', random_state=42)
kmeans_full.fit(X)
full_error = compute_quantization_error(X, kmeans_full.cluster_centers_)
full_error_normalized = full_error / n_samples
print("---------------FULL---------------")
print(f"Quantization error on full dataset (k={k}): {full_error:.2f}")
print(f"Normalized quantization error on full dataset (k={k}): {full_error_normalized:.6f}")


# 2. Find centers from corset for the lightweight coreset
corset_centers = kmeans_plus_plus_on_corset(X_cs_lwc, w_cs_lwc, k)

# Compute quantization error on full X based on these centers
corset_error_full_X = compute_quantization_error(X, corset_centers)
corset_error_full_X_normalized = corset_error_full_X / n_samples
print("---------------Lightweight coreset---------------")
print(f"Quantization error on full dataset using corset clusters (k={k}) with the lightweight method: {corset_error_full_X:.2f}")
print(f"Normalized quantization error using corset clusters (k={k}) with the lightweight method: {corset_error_full_X_normalized:.6f}")

# 3. Find centers from corset for the rbfnn coreset
X_cs_rbfnn_via_idx = X[idx]
corset_centers = kmeans_plus_plus_on_corset(X_cs_rbfnn_via_idx, w_cs_rbfnn, k)

# Compute quantization error on full X based on these centers
corset_error_full_X = compute_quantization_error(X, corset_centers)
corset_error_full_X_normalized = corset_error_full_X / n_samples
print("---------------RBFNN corset---------------")
print(f"Quantization error on full dataset using corset clusters (k={k}) with the RBFNN method: {corset_error_full_X:.2f}")
print(f"Normalized quantization error using corset clusters (k={k}) with the RBFNN method: {corset_error_full_X_normalized:.6f}")

# 4. Find centers from corset for the PCA dataset
kmeans_reduced = KMeans(n_clusters=k, init='k-means++', random_state=42)
kmeans_reduced.fit(X_reduced)
cluster_centers_original = pca.inverse_transform(kmeans_reduced.cluster_centers_)
full_error = compute_quantization_error(X, cluster_centers_original)
full_error_normalized = full_error / n_samples
print("---------------PCA---------------")
print(f"Quantization error on full dataset (k={k}): {full_error:.2f}")
print(f"Normalized quantization error on full dataset (k={k}): {full_error_normalized:.6f}")

---------------FULL---------------
Quantization error on full dataset (k=10): 402018.09
Normalized quantization error on full dataset (k=10): 0.691927
---------------Lightweight coreset---------------
Quantization error on full dataset using corset clusters (k=10) with the lightweight method: 394240.50
Normalized quantization error using corset clusters (k=10) with the lightweight method: 0.678541
---------------RBFNN corset---------------
Quantization error on full dataset using corset clusters (k=10) with the RBFNN method: 414741.81
Normalized quantization error using corset clusters (k=10) with the RBFNN method: 0.713827
---------------PCA---------------
Quantization error on full dataset (k=10): 504516.65
Normalized quantization error on full dataset (k=10): 0.868341
